### A: Persiapan Data untuk LSTM

In [2]:
# --- 1. Impor Semua Library yang Dibutuhkan ---
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# --- 2. Memuat dan Memilih Fitur ---
file_path_features = r'C:\MyFolder\Git\TA_SpatioTemporal\Data\parquet\dataset_final_features_energi_suhu.parquet'
df_lstm_input = pd.read_parquet(file_path_features)
df_lstm_input.dropna(inplace=True)

# Untuk LSTM, kita mulai dengan fitur numerik yang paling penting saja agar tidak terlalu kompleks
# Kita akan memasukkan 'meter_id' kembali nanti
FEATURES = [
    'konsumsi_energi', # Target kita juga menjadi fitur untuk membuat sekuens
    'konsumsi_lag_1_jam',
    'konsumsi_lag_24_jam',
    'avg_temp_previous_hour',
    'jam',
    'hari_minggu',
    'apakah_jam_kerja'
]
df_lstm_input = df_lstm_input[['meter_id', 'timestamp'] + FEATURES]

print("Data untuk LSTM berhasil dimuat.")

# --- 3. Normalisasi Data ---
# Neural network bekerja paling baik jika semua fitur diskalakan antara 0 dan 1.
scaler = MinMaxScaler()
# Kita skalakan hanya kolom fitur numerik
df_lstm_input[FEATURES] = scaler.fit_transform(df_lstm_input[FEATURES])

print("\nData berhasil dinormalisasi.")
print(df_lstm_input.head())

# --- 4. Fungsi untuk Membuat Sekuens (Windowing) ---
# Ini adalah fungsi inti untuk persiapan data LSTM
def create_sequences(data, n_past, n_future, target_col_index):
    """
    Mengubah data time series menjadi format sekuens untuk model LSTM.
    n_past: jumlah jam di masa lalu untuk dijadikan input.
    n_future: jumlah jam di masa depan untuk diprediksi (kita gunakan 1).
    target_col_index: index dari kolom target (konsumsi_energi).
    """
    X, y = [], []
    for i in range(n_past, len(data) - n_future + 1):
        X.append(data[i - n_past:i, :])
        y.append(data[i + n_future - 1:i + n_future, target_col_index])
    return np.array(X), np.array(y)

# --- 5. Terapkan Fungsi Sekuens per Gedung ---
# Kita harus membuat sekuens secara terpisah untuk setiap gedung
all_X, all_y = [], []
n_timesteps = 24  # Model akan melihat 24 jam ke belakang untuk memprediksi 1 jam ke depan
n_future = 1      # Memprediksi 1 jam ke depan

print(f"\nMembuat sekuens dengan melihat {n_timesteps} jam ke belakang...")

for meter in df_lstm_input['meter_id'].unique():
    # Ambil data untuk satu gedung
    gedung_data = df_lstm_input[df_lstm_input['meter_id'] == meter][FEATURES].values
    
    # Buat sekuens untuk gedung ini
    X_gedung, y_gedung = create_sequences(gedung_data, n_timesteps, n_future, target_col_index=0)
    
    all_X.append(X_gedung)
    all_y.append(y_gedung)

# Gabungkan hasil dari semua gedung
X_sequences = np.concatenate(all_X)
y_sequences = np.concatenate(all_y)

print("\nPembuatan sekuens selesai.")
print(f"Bentuk data input (X): {X_sequences.shape}") # (jumlah sampel, jam ke belakang, jumlah fitur)
print(f"Bentuk data target (y): {y_sequences.shape}")


Data untuk LSTM berhasil dimuat.

Data berhasil dinormalisasi.
   meter_id           timestamp  konsumsi_energi  konsumsi_lag_1_jam  \
24    BSC A 2024-06-02 00:00:00         0.066347            0.066619   
25    BSC A 2024-06-02 01:00:00         0.065227            0.066347   
26    BSC A 2024-06-02 02:00:00         0.064165            0.065227   
27    BSC A 2024-06-02 03:00:00         0.064281            0.064165   
28    BSC A 2024-06-02 04:00:00         0.064731            0.064281   

    konsumsi_lag_24_jam  avg_temp_previous_hour       jam  hari_minggu  \
24             0.067487                0.326593  0.000000          1.0   
25             0.068094                0.293468  0.043478          1.0   
26             0.067262                0.268432  0.086957          1.0   
27             0.067268                0.242853  0.130435          1.0   
28             0.066485                0.229140  0.173913          1.0   

    apakah_jam_kerja  
24               0.0  
25           

## Pembagian Data dan Pelatihan Model LSTM

In [3]:
# --- 1. Pembagian Data Sekuens ---
# Kita bagi secara sederhana, misal 80% untuk latih, 20% untuk uji
train_size = int(len(X_sequences) * 0.8)
X_train, X_test = X_sequences[:train_size], X_sequences[train_size:]
y_train, y_test = y_sequences[:train_size], y_sequences[train_size:]

print(f"\nData sekuens berhasil dibagi:")
print(f"Bentuk X_train: {X_train.shape}, Bentuk y_train: {y_train.shape}")
print(f"Bentuk X_test: {X_test.shape}, Bentuk y_test: {y_test.shape}")

# --- 2. Membangun Arsitektur Model LSTM ---
model_lstm = Sequential([
    # Input layer harus tahu bentuk data kita: (jam ke belakang, jumlah fitur)
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    # Layer LSTM dengan 50 unit memori
    LSTM(50, activation='relu'),
    # Output layer dengan 1 neuron untuk memprediksi 1 nilai (konsumsi_energi)
    Dense(1)
])

# Compile model
model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.summary()

# --- 3. Melatih Model ---
print("\nMulai melatih model LSTM...")
history = model_lstm.fit(
    X_train, y_train,
    epochs=20,  # Jumlah iterasi pelatihan
    batch_size=32,
    validation_split=0.1, # Gunakan 10% data latih untuk validasi internal
    verbose=1
)
print("\nModel LSTM berhasil dilatih!")



Data sekuens berhasil dibagi:
Bentuk X_train: (264825, 24, 7), Bentuk y_train: (264825, 1)
Bentuk X_test: (66207, 24, 7), Bentuk y_test: (66207, 1)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        11,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,651 (45.51 KB)

 Trainable params: 11,651 (45.51 KB)

 Non-trainable params: 0 (0.00 B)


Mulai melatih model LSTM...
Epoch 1/20
7449/7449 ━━━━━━━━━━━━━━━━━━━━ 53s 7ms/step - loss: 6.1700e-04 - val_loss: 8.6558e-04
Epoch 2/20
7449/7449 ━━━━━━━━━━━━━━━━━━━━ 48s 6ms/step - loss: 1.0629e-04 - val_loss: 8.4524e-04
Epoch 3/20
7449/7449 ━━━━━━━━━━━━━━━━━━━━ 50s 7ms/step - loss: 9.6751e-05 - val_loss: 8.0499e-04
Epoch 4/20
7449/7449 ━━━━━━━━━━━━━━━━━━━━ 49s 7ms/step - loss: 9.3415e-05 - val_loss: 7.6822e-04
Epoch 5/20
7449/7449 ━━━━━━━━━━━━━━━━━━━━ 62s 8ms/step - loss: 8.7933e-05 - val_loss: 8.1803e-04
Epoch 6/20
7449/7449 ━━━━━━━━━━━━━━━━━━━━ 61s 8ms/step - loss: 8.3184e-05 - val_loss: 8.1841e-04
Epoch 7/20
7449/7449 ━━━━━━━━━━━━━━━━━━━━ 61s 8ms/step - loss: 8.2297e-05 - val_loss: 8.0844e-04
Epoch 8/20
7449/7449 ━━━━━━━━━━━━━━━━━━━━ 54s 7ms/step - loss: 8.0827e-05 - val_loss: 7.4301e-04
Epoch 9/20
7449/7449 ━━━━━━━━━━━━━━━━━━━━ 55s 7ms/step - loss: 7.8378e-05 - val_loss: 8.0689e-04
Epoch 10/20
7449/7449 ━━━━━━━━━━━━━━━━━━━━ 54s 7ms/step - loss: 7.7356e-05 - val_loss: 7.4928e-04


## C: Evaluasi Kinerja Model LSTM

In [4]:
# --- 1. Membuat Prediksi ---
predictions_scaled = model_lstm.predict(X_test)

# --- 2. Inverse Transform (Mengembalikan ke Skala Asli) ---
# Kita harus membuat "dummy array" dengan bentuk yang sama seperti saat kita melakukan fit_transform
# agar bisa mengembalikan hanya kolom target kita.
dummy_array_pred = np.zeros((len(predictions_scaled), len(FEATURES)))
dummy_array_pred[:, 0] = predictions_scaled.flatten()
predictions_inversed = scaler.inverse_transform(dummy_array_pred)[:, 0]

dummy_array_true = np.zeros((len(y_test), len(FEATURES)))
dummy_array_true[:, 0] = y_test.flatten()
y_test_inversed = scaler.inverse_transform(dummy_array_true)[:, 0]

# --- 3. Hitung Metrik Evaluasi pada Data Asli ---
def calculate_smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.divide(numerator, denominator, out=np.zeros_like(denominator), where=denominator!=0)
    return np.mean(ratio) * 100

mae_lstm = mean_absolute_error(y_test_inversed, predictions_inversed)
rmse_lstm = np.sqrt(mean_squared_error(y_test_inversed, predictions_inversed))
r2_lstm = r2_score(y_test_inversed, predictions_inversed)
smape_lstm = calculate_smape(y_test_inversed, predictions_inversed)

print("\n--- Hasil Evaluasi Model LSTM ---")
print(f"Mean Absolute Error (MAE): {mae_lstm:.2f}")
print(f"Root Mean Square Error (RMSE): {rmse_lstm:.2f}")
print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape_lstm:.2f}%")
print(f"R-squared (R²): {r2_lstm:.2f}")


2069/2069 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step

--- Hasil Evaluasi Model LSTM ---
Mean Absolute Error (MAE): 1.00
Root Mean Square Error (RMSE): 2.54
Symmetric Mean Absolute Percentage Error (sMAPE): 64.00%
R-squared (R²): 0.95
